In [1]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rn
import time 
from sklearn.svm import SVC
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from scipy import linalg
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, pairwise
from sklearn.cluster import KMeans
from scipy.io import loadmat
from collections import Counter

d:\ryan\uni\fyp\venv3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


Import features

In [2]:
import json
with open('feature_data.json', 'r') as f:
    features = json.load(f)

features_ar = np.array(features)

# scaler = StandardScaler()
# scaler.fit(features_ar)
# features_ar = scaler.transform(features_ar)

In [3]:
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camIds = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelists = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()

train_idxss = np.subtract(train_idxs, 1)
gallery_idxss = np.subtract(gallery_idxs, 1)
query_idxss = np.subtract(query_idxs, 1)

In [4]:
camIds_train = camIds[train_idxss]
filelists_train = filelists[train_idxss]
labels_train = labels[train_idxss]
features_train = features_ar[train_idxss]

camIds_gal = camIds[gallery_idxss]
filelists_gal = filelists[gallery_idxss]
labels_gal = labels[gallery_idxss]
features_gal = features_ar[gallery_idxss]

camIds_query = camIds[query_idxss]
filelists_query = filelists[query_idxss]
labels_query = labels[query_idxss]
features_query = features_ar[query_idxss]

baseline kNN classifier

kNN with ranklist classification (baseline)

In [5]:
def rankKeval(labels_neigh, label_query):
    contained = False
    for i in range (0, len(labels_neigh)):
        if (labels_neigh[i] == label_query):
            contained = True
    return(contained)

In [6]:
def ret_NN(knn, query_l, query_f, query_Id, k, s_net, camIds_gal):
    n = k + s_net
    neigh = knn.kneighbors([query_f], n, return_distance=False) 
    neigh = neigh[0]
    m = 0
    while m<len(neigh):
        if ((query_l == labels_gal[neigh[m]]) and (query_Id == camIds_gal[neigh[m]])):
            neigh = np.delete(neigh, [m])
            m = m-1
        m = m+1
    m = len(neigh)
    if m>k:
        while m>k:
            neigh = np.delete(neigh, m-1)
            m = len(neigh)       
    labels_neigh = [labels_gal[i] for i in neigh]
    return (labels_neigh)

In [7]:
def scores_query(ranklist, query_l, instances, k):
    acc = 0
    TP = 0
    FP = 0
    prec = 0
    rec = 0
    
    if rankKeval(ranklist, query_l) == True:
        acc = 1
        
    for i in range (0, k):
        if (ranklist[i] == query_l):
            TP = TP + 1
        else:
            FP = FP +1
    prec = TP/(TP + FP)
    rec = TP/instances
    
    return(acc, prec, rec)

In [8]:
def scores_k(acc_l, prec_l, rec_l, k, query_n):
    acc_avg = []
    prec_avg = []
    rec_avg = []
    acc_avg.append(sum(acc_l)/query_n)
    prec_avg.append(sum(prec_l)/query_n)
    rec_avg.append(sum(rec_l)/query_n)
    return(acc_avg, prec_avg, rec_avg)

In [9]:
def average(lst, lstlen):
    avg = 0
    for i in range(0, lstlen-1):
        avg = avg + lst[i][0]
    avg = avg/lstlen
    return(avg)

In [10]:
def build_kNN(features, labels, k, s_net):
    nn= k + s_net
    knn = KNeighborsClassifier(n_neighbors=nn, p=2)
    knn.fit(features, labels)
    return(knn)

In [11]:
def label_instances(labels_g, labels_q, Ids_g, Ids_q):
#     construct a empty set of length n_query
#     go through allquery labels
#     see how many query labels of the same type but different camera exist in the gal set
    n_q = len(Ids_q)
    n_g = len(Ids_g)
    instances = [0]*n_q
    for i in range(0, n_q):
        for m in range(0, n_g):
            if (labels_g[m] == labels_q[i]) and not (Ids_g[m] == Ids_q[i]):
                instances[i] = instances[i] + 1
    return(instances)

In [12]:
def do_kNN(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k, s_net, instances):
    knn = build_kNN(features_gal, labels_gal, k, s_net)
    query_n = len(labels_query)
    print(query_n)
    acc = [0] * (query_n * k)
    prec = [0] * (query_n * k)
    rec = [0] * (query_n * k)
    AA = [0] * k
    AR = [0] * k 
    AP = [0] * k
    F1 = [0] * k
    mAP = [0] * k 
    for i in range (0, query_n):
        query_l = labels_query[i]
        query_f = features_query[i]
        query_Id = camIds_query[i]
        neigh = ret_NN(knn, query_l, query_f, query_Id, k, s_net, camIds_gal)
        for z in range (0, k):
            ind = i+(z*query_n)
            acc[ind], prec[ind], rec[ind] = scores_query(neigh, query_l, instances[i], (k-z))

            neigh.pop()
    for i in range (0, k):
        LB = i*query_n
        UB = (i+1)*query_n
        AA[i], AP[i], AR[i] = scores_k(acc[LB:UB], prec[LB:UB], rec[LB:UB], i+1, query_n)
        F1[i] = 2*(AP[i][0]*AR[i][0])/(AP[i][0]+AR[i][0])
        print(sum(AP[0]))
    mAP[i] = 1
    return(AA, AR, AP, mAP, F1)

In [13]:
AA, AR, AP, mAP = do_kNN(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 10, 9)

TypeError: do_kNN() missing 1 required positional argument: 'instances'

In [ ]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)